In [1]:
import numpy as np
import tifffile as tiff

import napari

from skimage.measure import manders_coloc_coeff, regionprops, label
from skimage.filters import threshold_otsu
from scipy.ndimage import zoom

from scripts.utils.image_utils import load_tiff_as_hyperstack


In [3]:
def calculate_manders_coefficient_3d(mask_stack, channel_stack):
        
    # Label the mask stack
    labels = label(mask_stack)
    props = regionprops(labels)
    
    # Initiate Manders' coefficients dictionary and Manders' coefficient based colored stack 
    manders_results = {}
    mask_colored_stack = np.zeros_like(anatomy_mask, dtype=float)
    
    # Binarize channel stack
    channel_threshold = threshold_otsu(channel_stack)
    binary_channel_stack = channel_stack > channel_threshold
    
    # Binarize the mask (assuming it's already binary)
    binary_mask_stack = anatomy_mask > 0
    
    for prop in props:
        # Get the bounding box of the region
        minr, minc, minz, maxr, maxc, maxz = prop.bbox
        
        # Extract the corresponding region from the red channel and the mask
        channel_region = binary_channel_stack[minr:maxr, minc:maxc, minz:maxz]
        mask_region = binary_mask_stack[minr:maxr, minc:maxc, minz:maxz]
                
        # Calculate Manders' colocalization coefficients
        manders_coeff = manders_coloc_coeff(mask_region, channel_region)
        
        # Store the results
        manders_results[prop.label] = manders_coeff
        
        # Color the stack based on the Manders' coefficients
        mask_colored_stack[minr:maxr, minc:maxc, minz:maxz][mask_region] = manders_coeff
        
    return manders_results, mask_colored_stack

In [6]:
# Define paths
anatomy_stack_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\raw\20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif'
anatomy_mask_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\pycpd\em_stack_channel 1_xfm_0_flfov_lmresolution_cp_masks.tif'

# Load anatomy stack
anatomy_stack = load_tiff_as_hyperstack(anatomy_stack_path, n_channels=2)
print(anatomy_stack.shape)

# Load anatomy mask
anatomy_mask = tiff.imread(anatomy_mask_path)

\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\raw\20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif loaded.
(2, 150, 512, 512)


In [8]:
# Flip and resample channel anatomy stack to fit mask stack (if necessary)
flipped_anatomy_stack = anatomy_stack[:, :, :, ::-1]

zoom_factors = [2.5, 1, 1]

# Initialize an empty array for the resampled stack with the correct shape
resampled_shape = (flipped_anatomy_stack.shape[0], int(flipped_anatomy_stack.shape[1] * zoom_factors[0]), flipped_anatomy_stack.shape[2], flipped_anatomy_stack.shape[3])
resampled_anatomy_stack = np.zeros(resampled_shape, dtype=flipped_anatomy_stack.dtype)

# Apply the zoom function to each channel
for channel in range(flipped_anatomy_stack.shape[0]):
    resampled_anatomy_stack[channel] = zoom(flipped_anatomy_stack[channel], zoom_factors, order=3)

# Save the resampled stack as a TIFF file
tiff.imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\processed\resampled_flipped_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif", resampled_anatomy_stack)
print("saved")


saved


In [10]:

# or updload already processed stack 
resampled_anatomy_stack_path  = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\processed\resampled_flipped_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif"
resampled_anatomy_stack2 = tiff.imread(resampled_anatomy_stack_path)
print(resampled_anatomy_stack2.shape)

(2, 375, 512, 512)


In [11]:
#Select channels 
c0_anatomy_stack = resampled_anatomy_stack[0] # Adjust if needed
c1_anatomy_stack = resampled_anatomy_stack[1] # Adjust if needed

# Calculate Manders' coefficients for each label and each channel
manders_results_c0, mask_colored_stack_c0 = calculate_manders_coefficient_3d(anatomy_mask, c0_anatomy_stack)
manders_results_c1, mask_colored_stack_c1 = calculate_manders_coefficient_3d(anatomy_mask, c1_anatomy_stack)

# Display the colored stacks
viewer = napari.Viewer()

viewer.add_image(mask_colored_stack_c0, name='mask_colored_stack_c0', colormap='green',blending='additive')
viewer.add_image(mask_colored_stack_c1, name='mask_colored_stack_c1', colormap='red',blending='additive')
viewer.add_image(c0_anatomy_stack, name='c0_anatomy_stack',blending='additive')
viewer.add_image(c1_anatomy_stack, name='c1_anatomy_stack',blending='additive')
viewer.add_image(anatomy_mask, name='anatomy_mask', blending='additive' )


<Image layer 'anatomy_mask' at 0x275f5838e20>

In [12]:
# Save manders_coeff based mask stack 

tiff.imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\masks\mask_manderscoeff_c0_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif", mask_colored_stack_c0)

tiff.imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\masks\mask_manderscoeff_c1_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif", mask_colored_stack_c1)